In [2]:
from google.colab import drive
drive.mount('/content/drive')
datadir = "/content/drive/My Drive/cs445_final/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#imports

import os
import math

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive




In [4]:
!pip install -q py-feat
# pyfeat pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have nump

In [5]:
from feat import Detector


In [13]:
detector = Detector(
    face_model="retinaface",
    landmark_model="mobilefacenet",
    au_model='xgb',
    emotion_model="resmasknet",
    facepose_model="img2pose"
)

# given default from pyfeat website

'''
im_file = datadir + 'images/subject01_happy.png'
im = np.float32(cv2.imread(im_file, cv2.IMREAD_GRAYSCALE) / 255.0)
plt.imshow(im, cmap='gray')
'''



data_dir = '/content/drive/MyDrive/cs445_final'
input_dir = os.path.join(data_dir, 'images')
output_dir = os.path.join(data_dir, 'cropped')

emotion_res = [] # store the reuslts from emotions


for file_name in os.listdir(input_dir):
  if file_name.endswith(('.png', '.jpg', '.jpeg')) and ("happy" in file_name or "sad" in file_name or "normal" in file_name): # only looking at those 3 files and no gif types

    file_path = os.path.join(input_dir, file_name)
    im = np.float32(cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) / 255.0)

    singular_face = detector.detect_image(file_path)
    if singular_face.faceboxes.values.size > 0: # if exists
        face_coords = singular_face.faceboxes.values[0] # get the coordinates, from the pyfeat result
        x, y, width, height = map(int, face_coords[:4])
        cropped_face = im[y:y+height, x:x+width]
        cropped_face = 255 * cropped_face # normalize


        core, ending = os.path.splitext(file_name) # add cropped to the names
        new_name = f"{core}_cropped{ending}"
        total_output = os.path.join(output_dir, new_name)

        # Save the cropped face image
        cv2.imwrite(total_output, cropped_face)


        data = singular_face.emotions # get the emotion output

        emotions_df = pd.DataFrame(data) # get data frame
        max_emotion = emotions_df.idxmax(axis=1)[0] # find the one with largest value
        print(max_emotion) # testing it on subject 1 happy
        emotion_res.append((file_name, max_emotion)) # store the emotion


results_df = pd.DataFrame(emotion_res, columns=['Filename', 'Max Emotion']) # create csv
results_df.to_csv(os.path.join(data_dir, 'emotion_results.csv'), index=False) # fill csv



100%|██████████| 1/1 [00:05<00:00,  5.86s/it]


happiness


100%|██████████| 1/1 [00:05<00:00,  5.48s/it]


neutral


100%|██████████| 1/1 [00:05<00:00,  5.33s/it]


...

100%|██████████| 1/1 [00:06<00:00,  6.12s/it]


happiness


100%|██████████| 1/1 [00:06<00:00,  6.04s/it]


happiness


"\nsingle_face_prediction = detector.detect_image(im_file)\nfigs = single_face_prediction.plot_detections(poses=True)\nprint(figs)\nprint()\n\nif single_face_prediction.faceboxes.values.size > 0:\n    face_coords = single_face_prediction.faceboxes.values[0]\n    x, y, width, height = map(int, face_coords[:4])\n    print(x, y, width, height)\n    cropped_face = im[y:y+height, x:x+width]\n\n    # Display the cropped face\n    plt.imshow(cropped_face, cmap='gray')\n    plt.show()\n\n\n\n    # Join the new base name with the original extension\n\n\n    cropped_face = 255 * cropped_face\n    #cv2.imwrite('/content/drive/MyDrive/cs445_final/cropped/subject01_happy.png', cropped_face)\n\n\n\n    data = single_face_prediction.emotions\n\n    emotions_df = pd.DataFrame(data)\n    max_emotion = emotions_df.idxmax(axis=1)[0]\n    print(max_emotion)\n"